In [53]:
import pandas as pd
import glob
from datetime import datetime
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime as dt


df=pd.read_csv("/Users/manbubble/Downloads/RedditAnalize/BERT_PRE/AllDate.csv",encoding="ISO-8859-1")

In [54]:
df

,Author,Body,Comment ID,Publish Date,Score,sentiment
0,kentroccas,Bitcoin will rise in Easter like Jesus himself,dwlscr1,2018-04-01 09:32:14,113.0,0
1,tizzletl,She's going to end up with one of those chad b...,dwlsk7a,2018-04-01 09:36:18,112.0,0
2,oignonne,Man with bitcoin asks how he can ensure his so...,dwlss9p,2018-04-01 09:40:48,307.0,0
3,yParticle,The clocks may lag a bit because they're also ...,dwlukot,2018-04-01 10:16:38,170.0,0
4,MakeTotalDestr0i,The branch of feminism and identity politics t...,dwlzcn6,2018-04-01 11:52:20,33.0,0
...,...,...,...,...,...,...
38494,dope_danny,"The cryptobros in literal tears going, and i q...",hirdyjo,2021-10-31 22:13:00,77.0,0
38495,Taubzi,"Aika monta argumenttia, joihin olisi kiva tart...",hirf4w1,2021-10-31 22:24:14,120.0,0
38496,ra693425,Imagine bitcoin just shooting straight up all ...,hirfntp,2021-10-31 22:29:22,214.0,0
38497,No-Village7980,Lost the keys to 5-6 bitcoin change I had when...,hirj2r2,2021-10-31 22:59:50,239.0,-1


In [55]:
df['Publish Date'] = pd.to_datetime(df['Publish Date'])


In [42]:
nan=df[(df['Publish Date'] < dt.datetime(2018,4,1,0,00,00))]
# nan.groupby('Author',as_index=False).mean()
nan_list=nan.mean()
pd.isna(nan_list['sentiment'])
# lenで0のときはNANなのでNAnを入れる処理を入れる

/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/322436265.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  nan_list=nan.mean()


True

In [40]:
nan_2=df[(df['Publish Date'] >= dt.datetime(2019,5,3)) & (df['Publish Date'] < dt.datetime(2019,5,6))]
# 6時間後から　0時間前まで
nan_list_2=nan_2.mean()
nan_list_2['sentiment']
nan_list_2

/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3159550037.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  nan_list_2=nan_2.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3159550037.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  nan_list_2=nan_2.mean()


Score        60.055556
sentiment     0.011111
dtype: float64

In [79]:
cols = ['time', '6_hours_ago','3_hours_ago']
# 基準の時間と、そこからいくらさかのぼって平均をとったか
variables_df = pd.DataFrame(index=[], columns=cols)
variables_df

,time,6_hours_ago,3_hours_ago


In [80]:
dt_now = dt.datetime(2018,4,1,00,00,00)
dt_now

datetime.datetime(2018, 4, 1, 0, 0)

In [81]:
df

,Author,Body,Comment ID,Publish Date,Score,sentiment
0,kentroccas,Bitcoin will rise in Easter like Jesus himself,dwlscr1,2018-04-01 09:32:14,113.0,0
1,tizzletl,She's going to end up with one of those chad b...,dwlsk7a,2018-04-01 09:36:18,112.0,0
2,oignonne,Man with bitcoin asks how he can ensure his so...,dwlss9p,2018-04-01 09:40:48,307.0,0
3,yParticle,The clocks may lag a bit because they're also ...,dwlukot,2018-04-01 10:16:38,170.0,0
4,MakeTotalDestr0i,The branch of feminism and identity politics t...,dwlzcn6,2018-04-01 11:52:20,33.0,0
...,...,...,...,...,...,...
38494,dope_danny,"The cryptobros in literal tears going, and i q...",hirdyjo,2021-10-31 22:13:00,77.0,0
38495,Taubzi,"Aika monta argumenttia, joihin olisi kiva tart...",hirf4w1,2021-10-31 22:24:14,120.0,0
38496,ra693425,Imagine bitcoin just shooting straight up all ...,hirfntp,2021-10-31 22:29:22,214.0,0
38497,No-Village7980,Lost the keys to 5-6 bitcoin change I had when...,hirj2r2,2021-10-31 22:59:50,239.0,-1


In [82]:
while dt_now<=dt.datetime(2021,11,1,00,00,00):
    time_6 = dt_now+dt.timedelta(hours=-6)
    time_3 = dt_now+dt.timedelta(hours=-3)
    tmp_df_6=df[(df['Publish Date'] >= time_6) & (df['Publish Date'] < dt_now)]
    tmp_df_3=df[(df['Publish Date'] >= time_3) & (df['Publish Date'] < dt_now)]
    mean_6=tmp_df_3.mean()
    mean_3=tmp_df_6.mean()
    s6=mean_6['sentiment']
    s3=mean_3['sentiment']
    record = pd.Series([dt_now,s6,s3], index=variables_df.columns)
    variables_df=variables_df.append(record,ignore_index=True)
    dt_now=dt_now+dt.timedelta(hours=1)
    

/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/1524060518.py:6: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_6=tmp_df_3.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/1524060518.py:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_3=tmp_df_6.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/1524060518.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_6=tmp_df_3.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/1524060518.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') 

In [87]:
variables_df.to_csv("t3t6_mean.csv",index=False)
# Nanはそのままにしてる

In [84]:
print(variables_df.dropna(subset=['6_hours_ago']))

                     time 6_hours_ago 3_hours_ago
10    2018-04-01 10:00:00         0.0         0.0
11    2018-04-01 11:00:00         0.0         0.0
12    2018-04-01 12:00:00         0.0         0.0
13    2018-04-01 13:00:00         0.2       0.125
14    2018-04-01 14:00:00    0.166667         0.1
...                   ...         ...         ...
31436 2021-10-31 20:00:00       -0.25       -0.25
31437 2021-10-31 21:00:00         0.0        -0.2
31438 2021-10-31 22:00:00         0.0        -0.2
31439 2021-10-31 23:00:00        -0.2   -0.222222
31440 2021-11-01 00:00:00        -0.2   -0.111111

[22124 rows x 3 columns]


In [85]:
print(variables_df.dropna(subset=['3_hours_ago']))

                     time 6_hours_ago 3_hours_ago
10    2018-04-01 10:00:00         0.0         0.0
11    2018-04-01 11:00:00         0.0         0.0
12    2018-04-01 12:00:00         0.0         0.0
13    2018-04-01 13:00:00         0.2       0.125
14    2018-04-01 14:00:00    0.166667         0.1
...                   ...         ...         ...
31436 2021-10-31 20:00:00       -0.25       -0.25
31437 2021-10-31 21:00:00         0.0        -0.2
31438 2021-10-31 22:00:00         0.0        -0.2
31439 2021-10-31 23:00:00        -0.2   -0.222222
31440 2021-11-01 00:00:00        -0.2   -0.111111

[24927 rows x 3 columns]


In [88]:
# 分散
cols = ['time', '6_hours_ago','3_hours_ago']
# 基準の時間と、そこからいくらさかのぼって平均をとったか
variables_df = pd.DataFrame(index=[], columns=cols)
dt_now = dt.datetime(2018,4,1,00,00,00)
while dt_now<=dt.datetime(2021,11,1,00,00,00):
    time_6 = dt_now+dt.timedelta(hours=-6)
    time_3 = dt_now+dt.timedelta(hours=-3)
    tmp_df_6=df[(df['Publish Date'] >= time_6) & (df['Publish Date'] < dt_now)]
    tmp_df_3=df[(df['Publish Date'] >= time_3) & (df['Publish Date'] < dt_now)]
    mean_6=tmp_df_3.var()
    mean_3=tmp_df_6.var()
    s6=mean_6['sentiment']
    s3=mean_3['sentiment']
    record = pd.Series([dt_now,s6,s3], index=variables_df.columns)
    variables_df=variables_df.append(record,ignore_index=True)
    dt_now=dt_now+dt.timedelta(hours=1)

/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/696831427.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_6=tmp_df_3.var()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/696831427.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_3=tmp_df_6.var()


In [89]:
variables_df

,time,6_hours_ago,3_hours_ago
0,2018-04-01 00:00:00,NaN,NaN
1,2018-04-01 01:00:00,NaN,NaN
2,2018-04-01 02:00:00,NaN,NaN
3,2018-04-01 03:00:00,NaN,NaN
4,2018-04-01 04:00:00,NaN,NaN
...,...,...,...
31436,2021-10-31 20:00:00,0.25,0.25
31437,2021-10-31 21:00:00,0.0,0.2
31438,2021-10-31 22:00:00,0.0,0.2
31439,2021-10-31 23:00:00,0.2,0.194444


In [90]:
variables_df.to_csv("t3t6_var.csv",index=False)
# Nanはそのままにしてる

In [91]:
cols = ['time', '6_hours_ago','3_hours_ago']
# 基準の時間と、そこからいくらさかのぼって平均をとったか
variables_df = pd.DataFrame(index=[], columns=cols)
dt_now = dt.datetime(2018,4,1,00,00,00)
df_abs=df
df_abs['sentiment']=df_abs['sentiment'].abs()
df_abs
while dt_now<=dt.datetime(2021,11,1,00,00,00):
    time_6 = dt_now+dt.timedelta(hours=-6)
    time_3 = dt_now+dt.timedelta(hours=-3)
    tmp_df_6=df_abs[(df_abs['Publish Date'] >= time_6) & (df_abs['Publish Date'] < dt_now)]
    tmp_df_3=df_abs[(df_abs['Publish Date'] >= time_3) & (df_abs['Publish Date'] < dt_now)]
    mean_6=tmp_df_3.mean()
    mean_3=tmp_df_6.mean()
    s6=mean_6['sentiment']
    s3=mean_3['sentiment']
    record = pd.Series([dt_now,s6,s3], index=variables_df.columns)
    variables_df=variables_df.append(record,ignore_index=True)
    dt_now=dt_now+dt.timedelta(hours=1)

/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3736515195.py:13: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_6=tmp_df_3.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3736515195.py:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_3=tmp_df_6.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3736515195.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_6=tmp_df_3.mean()
/var/folders/82/60v49wn11l51dmpxrl2yk7m80000gn/T/ipykernel_10497/3736515195.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=Non

In [92]:
variables_df

,time,6_hours_ago,3_hours_ago
0,2018-04-01 00:00:00,NaN,NaN
1,2018-04-01 01:00:00,NaN,NaN
2,2018-04-01 02:00:00,NaN,NaN
3,2018-04-01 03:00:00,NaN,NaN
4,2018-04-01 04:00:00,NaN,NaN
...,...,...,...
31436,2021-10-31 20:00:00,0.25,0.25
31437,2021-10-31 21:00:00,0.0,0.2
31438,2021-10-31 22:00:00,0.0,0.2
31439,2021-10-31 23:00:00,0.2,0.222222


In [93]:
variables_df.to_csv("t3t6_abs_mean.csv",index=False)